In [26]:
import os
import openai
from dotenv import load_dotenv

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.language_model import BaseLanguageModel
from langchain.agents import Tool
from langchain.agents import ZeroShotAgent
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.tools import ShellTool
from langchain.python import PythonREPL

# Reomve line if needs to be set to false
os.environ["VERBOSE"] = "true"
load_dotenv()

True

Tools
=====

In [27]:
shell_tool = ShellTool()
python_tool = PythonREPLTool()


#shell_tool.description = shell_tool.description + f"args {shell_tool.args}".replace("{", "{{").replace("}", "}}")
print(shell_tool.description)

tools = [
    shell_tool,
    python_tool
]

Run shell commands on this Linux machine.


Todo chain
----------

In [28]:

# Create a chain thats only goal is to set up the initial todo-list
todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}. Keep it simple, the fewer tasks the better. Do not install anything!"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)

In [29]:
# Parser extracts a todo list from the llm output
import re
def extract_todolist(text):
    pattern = r'\n\d+\.\s(.*?)(?=\n|$)'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches

Agent
-----

In [33]:
objective = "Scan the current network."

prefix = """You are an AI who performs a single task based on the following objective: '{objective}'. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=['objective', 'context', 'task']
)

llm = ChatOpenAI(temperature=0.5, model="gpt-4")
print(type(llm))

agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

<class 'langchain.chat_models.openai.ChatOpenAI'>


Main
----

In [32]:
completed_tasks = []
context = ""
task_list = extract_todolist(todo_chain.run(objective))

for i, task in enumerate(task_list):
    print(f"{i}: {task}")


#tmp
task_list.pop(1)

while len(task_list) > 0:
    print("-------------------------- new task -----------------------------")
    print(context)
    print(task_list[0])
    input("Press enter to continue with next task")

    output = agent_executor.run(
        prompt.format(
            task=task_list[0],
            objective=objective,
            context=context
        )
    )
    print(output)

    completed_tasks.append(task_list.pop(0))

    context += f"Question: {completed_tasks[-1]}\nFinal Answer: {output}\n"


0: Gather information about the current network (IP addresses, devices, etc.)
1: Create a network diagram of the current network
2: Perform a port scan of the network
3: Perform a vulnerability scan of the network
4: Analyze the results of the scans and identify any potential security issues
5: Document the findings and recommendations for remediation
-------------------------- new task -----------------------------

Gather information about the current network (IP addresses, devices, etc.)


> Entering new AgentExecutor chain...
Thought: To gather information about the current network, I can use the `ifconfig` command in terminal, which will give me details about the network interfaces. 

Action: 
```
{
  "action": "terminal",
  "action_input": "ifconfig"
}
```
Observation: enp5s0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 192.168.1.179  netmask 255.255.255.0  broadcast 192.168.1.255
        inet6 fd57:a00f:f8bb:4486:92ee:528:c4e7:794b  prefixlen 64  scopeid 0x

KeyboardInterrupt: 

In [34]:
import langchain
print(langchain.Request())

AttributeError: module 'langchain' has no attribute 'Request'

In [35]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

print(StreamingStdOutCallbackHandler())